# Uniswap V3 limit price data decoding

When run from top to bottom, this notebook collects all transactions sent to the Universal Router contract and decodes the limit price data from the calldata. The data is stored in Azure blob storage and downloaded manually from Dune Analytics. The csv files are readable by anyone for reproducibility.

In [56]:
import os
import io
from dotenv import load_dotenv

import pandas as pd
from eth_abi import abi
from eth_utils import decode_hex, to_int, to_hex

from sqlalchemy import create_engine, String
from azure.storage.blob import BlobServiceClient

from tqdm import tqdm, trange

###

load_dotenv()

True

In [15]:
# Adjust the minimum with of a pandas column
pd.set_option('display.max_colwidth', None)

In [16]:
postgresql_uri_us = os.environ["POSTGRESQL_URI_US"]
postgresql_uri_mp = os.environ["POSTGRESQL_URI_MP"]
blobstorage_uri = os.environ["AZURE_STORAGE_CONNECTION_STRING"]


## Fetch data from the Azure storage bucket

Data is collected from Dune analytics with the following query:

```sql
select
  call_tx_hash, commands, inputs
from
  uniswap_ethereum.UniversalRouter_call_execute
where call_block_number <= 17864015
order by
  call_block_number desc
offset 0
limit 250000
```

In [19]:
blob_service_client = BlobServiceClient.from_connection_string(blobstorage_uri)

container_name = "uniswap-v3-universalrouter-call"
container_client = blob_service_client.get_container_client(container_name)

blobs_list = list(container_client.list_blobs())
len(blobs_list), blobs_list[:3]

(10,
 [{'name': '0.csv', 'container': 'uniswap-v3-universalrouter-call', 'snapshot': None, 'version_id': None, 'is_current_version': None, 'blob_type': <BlobType.BLOCKBLOB: 'BlockBlob'>, 'metadata': {}, 'encrypted_metadata': None, 'last_modified': datetime.datetime(2023, 8, 16, 13, 37, 18, tzinfo=datetime.timezone.utc), 'etag': '0x8DB9E5DE9778047', 'size': 231077169, 'content_range': None, 'append_blob_committed_block_count': None, 'is_append_blob_sealed': None, 'page_blob_sequence_number': None, 'server_encrypted': True, 'copy': {'id': None, 'source': None, 'status': None, 'progress': None, 'completion_time': None, 'status_description': None, 'incremental_copy': None, 'destination_snapshot': None}, 'content_settings': {'content_type': 'text/csv', 'content_encoding': None, 'content_language': None, 'content_md5': None, 'content_disposition': None, 'cache_control': None}, 'lease': {'status': 'unlocked', 'state': 'available', 'duration': None}, 'blob_tier': None, 'rehydrate_priority': No

In [20]:
df_all = None

for blob in tqdm(list(container_client.list_blobs())):
    assert blob and blob.name

    if not blob.name.endswith('.csv'):
        continue

    # download the blob to memory
    blob_bytes = container_client.download_blob(blob).readall()

    # convert the blob to a dataframe
    df = pd.read_csv(io.BytesIO(blob_bytes), encoding='utf-8', low_memory=False)

    # append the dataframe to the master dataframe
    if df_all is None:
        df_all = df
    else:
        df_all = pd.concat([df_all, df], ignore_index=True, axis=0)

assert df_all is not None

print(df_all.shape)

df_all.head()

100%|██████████| 10/10 [03:19<00:00, 19.99s/it]

(2500000, 3)


,call_tx_hash,commands,inputs
0,0xaaf48ba1efd20cd031d33e66a429ebe1168b5389034cee562e537840271cd788,0x0b090c,[0x0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000028e88b9e7c1410a 0x000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000009fdf42f6e480000000000000000000000000000000000000000000000000000028e88b9e7c1410a00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc200000000000000000000000021419383bbd94e4213dd7ca324b66ba8b81cc530 0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000]
1,0xd286f2ea2ab839088cad73c1402c2f2fe23d0c8fbbe138d76b8138fb3da624ea,0x0a080c,[0x000000000000000000000000ab98093c7232e98a47d7270ce0c1c2106f61c73b000000000000000000000000ffffffffffffffffffffffffffffffffffffffff0000000000000000000000000000000000000000000000000000000064f89dbe00000000000000000000000000000000000000000000000000000000000000000000000000000000000000003fc91a3afd70395cd496c647d5a6cc9d4b2b7fad0000000000000000000000000000000000000000000000000000000064d117c600000000000000000000000000000000000000000000000000000000000000e00000000000000000000000000000000000000000000000000000000000000041490303c39535d239f108de5e083cd9c497f2eae18a6e6b47903c1329d396c935494e595342c620bd4c3072e6479796df6ecce7b12967dbb5e7563c4214955fcf1c00000000000000000000000000000000000000000000000000000000000000 0x0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000070d6137d0cd300000000000000000000000000000000000000000000000000d9285026e0d83300000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000002000000000000000000000000ab98093c7232e98a47d7270ce0c1c2106f61c73b000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0x000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000d9285026e0d833]
2,0x8ead87a366e490f4d4bccdea8c88d7b986558a11a6129ce4870789192a2a0aef,0x0b08,[0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000009b6e64a8ec60000 0x000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000009b6e64a8ec600000000000000000000000000000000000000000000000000001f1a99254e52c13d00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc200000000000000000000000021419383bbd94e4213dd7ca324b66ba8b81cc530]
3,0x01344dde2cbf46f82b28e9fd545f818d850c52c6aaffefe75dd7657a71dd9fe4,0x080c,[0x0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000399614988b000000000000000000000000000000000000000000000000008c622fc689e27800000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000020000000000000000000000004f849c55180ddf8185c5cc495ed58c3aea9c9a28000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0x0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000008c622fc689e278]
4,0x26efe3924b80edeb439a6cd008e4efc300cd16735fa9a39fb159342d4c4d5e19,0x080c,[0x0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000de0b6b3a764000000000000000000000000000000

In [22]:
assert df_all is not None

df = df_all.drop_duplicates(keep='first')

df.shape

(2499980, 3)

## Decode the data from the binary format

In [32]:
search_commands = ['00', '01']
command_mapping = {"00": "V3_SWAP_EXACT_IN", 
                   "01": "V3_SWAP_EXACT_OUT"}

# vectorized clean the data
df['split_command'] = df['commands'].str.rstrip().str.split("").apply(lambda x: [_x for _x in x if _x != ""])
df['clean_commands'] = df['split_command'].apply(lambda x: [x[i] + x[i+1] for i in range(0, len(x), 2)])
df['clean_commands'] = df['clean_commands'].apply(lambda x: [_x for _x in x if _x != "0x"]).copy()
df['command_location'] = df['clean_commands'].apply(lambda x: [count for (count, row) in enumerate(x) 
                                                               if row in search_commands])

df['clean_inputs'] = df['inputs'].apply(lambda x: [_x.replace("[", "").replace("]", "") for _x in x.split(" ")])

v3_swaps = df[df['command_location'].apply(lambda x: len(x) != 0)].copy(); v3_swaps.head()

,call_tx_hash,commands,inputs,split_command,clean_commands,command_location,clean_inputs
5,0x868a4da8d302128e1614d620e68925927e55ca5672e1a7dfffadbd53501ce224,0x0b00,[0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000002f129cd2c705ee6 0x000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000002f129cd2c705ee60000000000000000000000000000000000000000006d4fc8c3a0f1d2ea257a4200000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002bc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2002710423f4e6138e475d85cf7ea071ac92097ed631eea000000000000000000000000000000000000000000],"[0, x, 0, b, 0, 0]","[0b, 00]",[1],"[0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000002f129cd2c705ee6, 0x000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000002f129cd2c705ee60000000000000000000000000000000000000000006d4fc8c3a0f1d2ea257a4200000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002bc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2002710423f4e6138e475d85cf7ea071ac92097ed631eea000000000000000000000000000000000000000000]"
8,0x707c06a31c4c8f91b6c142fb11ea685cfcdca781cdd79bce9298ceab02a3fb9d,0x0b00,[0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000009b6e64a8ec60000 0x000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000009b6e64a8ec6000000000000000000000000000000000000000000000166eb64e3b3df7300b5683500000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002bc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2002710423f4e6138e475d85cf7ea071ac92097ed631eea000000000000000000000000000000000000000000],"[0, x, 0, b, 0, 0]","[0b, 00]",[1],"[0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000009b6e64a8ec60000, 0x000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000009b6e64a8ec6000000000000000000000000000000000000000000000166eb64e3b3df7300b5683500000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002bc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2002710423f4e6138e475d85cf7ea071ac92097ed631eea000000000000000000000000000000000000000000]"
10,0x9cce38269a1b337084c4c31b73bac9bc75ae565b0ad0f43169605f81121ab28c,0x0b010c,[0x00000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000002a3d2d2a50b0733d 0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000147d357000000000000000000000000000000000000000000000000002a3d2d2a50b0733d00000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002ba0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000bb8c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2000000000000000000000000000000000000000000 0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000],"[0, x, 0, b, 0, 1, 0, c]","[0b, 01, 0c]",[1],"[0x00000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000002a3d2d2a50b0733d, 0x0000000000000000000000000000000000000000000000000

In [37]:
all_data = []
errors = 0
for i in trange(v3_swaps.shape[0]):
    swap = v3_swaps.iloc[i]

    for command_location in swap['command_location']:
        
        command = str(swap['clean_commands'][command_location])
        input_str = str(swap['clean_inputs'][command_location])
        input_str = decode_hex(input_str)

        try:
            decodedABI = abi.decode(['address', 'uint256', 'uint256', 'bytes', 'bool'], input_str)
        except:
            errors += 1
            continue

        path = decodedABI[3]

        payload = []
        offset = 43

        addr1 = to_hex(path[0:20])
        fee = to_int(path[20:23])
        addr2 = to_hex(path[23:43])
        payload.append([addr1, fee, addr2])

        while offset < len(path):
            # consume the path
            addr1 = addr2
            fee = to_int(path[offset: offset+3])
            addr2 = to_hex(path[offset+3: offset+23])

            payload.append([addr1, fee, addr2])

            offset += 23

        if len(path) < 43:
            # this can only happen if bug (i think)
            errors += 1
            continue
            # raise ValueError("Incorrect Path Specification")
        
        command_str = command_mapping[command]
        cleaned_input = {"recipient": decodedABI[0],
                    f"{'amountIn' if command == '00' else 'amountOut'}": decodedABI[1],
                    f"{'amountOutMin' if command == '00' else 'amountInMax'}": decodedABI[2],
                    "path": payload,
                    "payerIsUser": decodedABI[4]}
        
        all_data.append([command_str, cleaned_input, swap['call_tx_hash']])




for item in all_data[:3]:
    print(item)

len(all_data), f"Errors {errors}"

100%|██████████| 451332/451332 [00:26<00:00, 17160.09it/s]

['V3_SWAP_EXACT_IN', {'recipient': '0x0000000000000000000000000000000000000001', 'amountIn': 211996618653720294, 'amountOutMin': 132149684735488745768581698, 'path': [['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 10000, '0x423f4e6138e475d85cf7ea071ac92097ed631eea']], 'payerIsUser': False}, '0x868a4da8d302128e1614d620e68925927e55ca5672e1a7dfffadbd53501ce224']
['V3_SWAP_EXACT_IN', {'recipient': '0x0000000000000000000000000000000000000001', 'amountIn': 700000000000000000, 'amountOutMin': 433907060627622534592161845, 'path': [['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 10000, '0x423f4e6138e475d85cf7ea071ac92097ed631eea']], 'payerIsUser': False}, '0x707c06a31c4c8f91b6c142fb11ea685cfcdca781cdd79bce9298ceab02a3fb9d']
['V3_SWAP_EXACT_OUT', {'recipient': '0x0000000000000000000000000000000000000001', 'amountOut': 5500000000, 'amountInMax': 3043638582937940797, 'path': [['0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 3000, '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']], 'payerIsUser': False}, 

(466579, 'Errors 5')

In [49]:
# Take out transactions that are multi-hop as it's not straightforward to
# derive appropriate limit prices for intermediate hops
cleaned_data = [item for item in all_data if len(item[1]["path"]) == 1]

print(len(all_data), "-->", len(cleaned_data), f"{len(cleaned_data) / len(all_data):.0%}")

466579 --> 400995 86%


In [45]:
flattened_data = []

for item in tqdm(cleaned_data):
    transaction_type = item[0]
    transaction_details = item[1]
    transaction_hash = item[2]

    # Flatten path separately as it's a nested list
    paths = transaction_details['path']
    for path in paths:
        token0, fee, token1 = path
        flattened_data.append({
            'transaction_type': transaction_type,
            'transaction_hash': transaction_hash,
            'recipient': transaction_details['recipient'],
            'amountIn': transaction_details.get('amountIn'),
            'amountOut': transaction_details.get('amountOut'),
            'amountOutMin': transaction_details.get('amountOutMin'),
            'amountInMax': transaction_details.get('amountInMax'),
            'payerIsUser': transaction_details['payerIsUser'],
            'token0': token0,
            'fee': fee,
            'token1': token1,
        })

# Convert to DataFrame
df = pd.DataFrame(flattened_data, dtype='object'); print(df.shape); df.head()

100%|██████████| 400995/400995 [00:00<00:00, 1381508.10it/s]


(400995, 11)


,transaction_type,transaction_hash,recipient,amountIn,amountOut,amountOutMin,amountInMax,payerIsUser,token0,fee,token1
0,V3_SWAP_EXACT_IN,0x868a4da8d302128e1614d620e68925927e55ca5672e1a7dfffadbd53501ce224,0x0000000000000000000000000000000000000001,211996618653720294,None,132149684735488745768581698,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000,0x423f4e6138e475d85cf7ea071ac92097ed631eea
1,V3_SWAP_EXACT_IN,0x707c06a31c4c8f91b6c142fb11ea685cfcdca781cdd79bce9298ceab02a3fb9d,0x0000000000000000000000000000000000000001,700000000000000000,None,433907060627622534592161845,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000,0x423f4e6138e475d85cf7ea071ac92097ed631eea
2,V3_SWAP_EXACT_OUT,0x9cce38269a1b337084c4c31b73bac9bc75ae565b0ad0f43169605f81121ab28c,0x0000000000000000000000000000000000000001,None,5500000000,None,3043638582937940797,False,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3,V3_SWAP_EXACT_IN,0x0815280508f43c94e4a0a2991f24108124effac762c120022b9e66febbe88696,0x0000000000000000000000000000000000000002,5273000000,None,2882045483002369817,None,True,0xdac17f958d2ee523a2206206994597c13d831ec7,500,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
4,V3_SWAP_EXACT_IN,0x326e3cacb000b3dc63578710e619a268a2734373c75ecd1df31c4a259789b4d1,0x0000000000000000000000000000000000000002,200000000000000000000000,None,850173983075597316,None,True,0x0c572544a4ee47904d54aaa6a970af96b6f00e1b,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


## Get the initial set of transactions to focus on

In [48]:
# Take out transactions that combine with a Uniswap V2 pool as we don't
# have an easy way to get the limit prices for those swaps

df = df[(~(df.amountOutMin == 0))]
df = df[(~(df.amountInMax == 0))]

print(len(cleaned_data), "-->", df.shape[0], f"{df.shape[0] / len(cleaned_data):.0%}")

df.head()

400995 --> 386798 96%


,transaction_type,transaction_hash,recipient,amountIn,amountOut,amountOutMin,amountInMax,payerIsUser,token0,fee,token1
0,V3_SWAP_EXACT_IN,0x868a4da8d302128e1614d620e68925927e55ca5672e1a7dfffadbd53501ce224,0x0000000000000000000000000000000000000001,211996618653720294,None,132149684735488745768581698,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000,0x423f4e6138e475d85cf7ea071ac92097ed631eea
1,V3_SWAP_EXACT_IN,0x707c06a31c4c8f91b6c142fb11ea685cfcdca781cdd79bce9298ceab02a3fb9d,0x0000000000000000000000000000000000000001,700000000000000000,None,433907060627622534592161845,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000,0x423f4e6138e475d85cf7ea071ac92097ed631eea
2,V3_SWAP_EXACT_OUT,0x9cce38269a1b337084c4c31b73bac9bc75ae565b0ad0f43169605f81121ab28c,0x0000000000000000000000000000000000000001,None,5500000000,None,3043638582937940797,False,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3,V3_SWAP_EXACT_IN,0x0815280508f43c94e4a0a2991f24108124effac762c120022b9e66febbe88696,0x0000000000000000000000000000000000000002,5273000000,None,2882045483002369817,None,True,0xdac17f958d2ee523a2206206994597c13d831ec7,500,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
4,V3_SWAP_EXACT_IN,0x326e3cacb000b3dc63578710e619a268a2734373c75ecd1df31c4a259789b4d1,0x0000000000000000000000000000000000000002,200000000000000000000000,None,850173983075597316,None,True,0x0c572544a4ee47904d54aaa6a970af96b6f00e1b,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


## Put the transactions into the database

In [50]:
engine = create_engine(postgresql_uri_us)

In [51]:
pools = pd.read_sql_query(
    """
    SELECT * FROM factory
    """,
    engine,
)[['token0', 'token1', 'fee', "pool"]]

pools

,token0,token1,fee,pool
0,0x8d96b4ab6c741a4c8679ae323a100d74f085ba8f,0xdac17f958d2ee523a2206206994597c13d831ec7,100,0x5c2a6a370e63b4de95650edc565ad0fbab5312ae
1,0x4ad7a056191f4c9519facd6d75fa94ca26003ace,0xdac17f958d2ee523a2206206994597c13d831ec7,100,0x554b887c98fcfac8b3acb9e40a2ab723dc2d8370
2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xf1182229b71e79e504b1d2bf076c15a277311e05,100,0x996dae6bad0ce9eb1f4924953129429699d3c584
3,0x8eee1a04b3223d1667152f5fb148f6ea6420d755,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,100,0x1813316975d62fcd72e7beaff93d8c6e7f52cb8f
4,0x67639ca498a180b3933d01c806bc5cb49f7154d8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,100,0x78a1b525730cc939d59147221ee61924997a5be5
...,...,...,...,...
14336,0x5b7e436cc7652c32e327e62293350b212fb46f7c,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0x9bbbfbf3d6e3ef36550fee8fc61f909c9e1b43fc
14337,0x809e427fad772042e2507a144634aecbd559f7f1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0xa3524cc509338fff7ffa684b210ef8a03ad771de
14338,0x9c4f48122176c39fa2a57aebed337b7874f0266a,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0x05117772aee309f389d23f712fe4e135f3c0c0a0
14339,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xeb1c06e85f2c4c7115b87d57bf014c1dead06146,3000,0xc51cdedf2ab2ac5113f1c3de0876850278c625bc


In [52]:
join_key_func = lambda x: f"{x['token0']}-{x['token1']}-{x['fee']}" if x['token0'] <= x['token1'] else f"{x['token1']}-{x['token0']}-{x['fee']}"

pools = pools.assign(join_key=pools.apply(join_key_func, axis=1)).drop(columns=['token0', 'token1', 'fee'])

pools

,pool,join_key
0,0x5c2a6a370e63b4de95650edc565ad0fbab5312ae,0x8d96b4ab6c741a4c8679ae323a100d74f085ba8f-0xdac17f958d2ee523a2206206994597c13d831ec7-100
1,0x554b887c98fcfac8b3acb9e40a2ab723dc2d8370,0x4ad7a056191f4c9519facd6d75fa94ca26003ace-0xdac17f958d2ee523a2206206994597c13d831ec7-100
2,0x996dae6bad0ce9eb1f4924953129429699d3c584,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48-0xf1182229b71e79e504b1d2bf076c15a277311e05-100
3,0x1813316975d62fcd72e7beaff93d8c6e7f52cb8f,0x8eee1a04b3223d1667152f5fb148f6ea6420d755-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-100
4,0x78a1b525730cc939d59147221ee61924997a5be5,0x67639ca498a180b3933d01c806bc5cb49f7154d8-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-100
...,...,...
14336,0x9bbbfbf3d6e3ef36550fee8fc61f909c9e1b43fc,0x5b7e436cc7652c32e327e62293350b212fb46f7c-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-3000
14337,0xa3524cc509338fff7ffa684b210ef8a03ad771de,0x809e427fad772042e2507a144634aecbd559f7f1-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-3000
14338,0x05117772aee309f389d23f712fe4e135f3c0c0a0,0x9c4f48122176c39fa2a57aebed337b7874f0266a-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-3000
14339,0xc51cdedf2ab2ac5113f1c3de0876850278c625bc,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-0xeb1c06e85f2c4c7115b87d57bf014c1dead06146-3000


In [53]:
df = df.assign(join_key=df.apply(join_key_func, axis=1))

df_joined = df.merge(pools, on='join_key', how='inner').drop(columns=['join_key'])

df_joined

,transaction_type,transaction_hash,recipient,amountIn,amountOut,amountOutMin,amountInMax,payerIsUser,token0,fee,token1,pool
0,V3_SWAP_EXACT_OUT,0x9cce38269a1b337084c4c31b73bac9bc75ae565b0ad0f43169605f81121ab28c,0x0000000000000000000000000000000000000001,None,5500000000,None,3043638582937940797,False,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
1,V3_SWAP_EXACT_IN,0x15999fd52f3ed1727ec190f6fb4673450b2e0b62f199c7d1eceeca301d946182,0x0000000000000000000000000000000000000001,650000000000000000,None,1103352830,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
2,V3_SWAP_EXACT_IN,0x7222ecbbb6dd33521c05d478880c61ee3a4296c631b28c71f9d5a7c7111c3f34,0x0000000000000000000000000000000000000001,10000000000000000000,None,18140883228,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
3,V3_SWAP_EXACT_IN,0x2dd12bd8609a3bf9e302cc56e2e985922c10dca666b9a31946db672f7fdf0e54,0x0000000000000000000000000000000000000001,2200000000000000000,None,3784056538,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
4,V3_SWAP_EXACT_IN,0x52d534f83c27061d2b8889c9f3068f650f3dda2bd1ba4c41fc923bc6bfa747c2,0x0000000000000000000000000000000000000001,3533224512882063209,None,6409780921,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
...,...,...,...,...,...,...,...,...,...,...,...,...
290966,V3_SWAP_EXACT_IN,0xfb3cde48c8c90c4280eacd873e4b51823afab554ae8f13dd920fdebf7e3cb494,0x0000000000000000000000000000000000000002,1100000000000000000000,None,1337676355630739,None,True,0xb22c05cedbf879a661fcc566b5a759d005cf7b4c,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x6d1f6aac7c375b66d87026aea2d3e04d760e624d
290967,V3_SWAP_EXACT_OUT,0x1f881a978c5faa5efdbc57f75e37b923127acec7291007e6c1d0700a64315500,0x0000000000000000000000000000000000000001,None,200,None,1134685552320951,False,0x056fd409e1d7a124bd7017459dfea2f387b6d5cd,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x4ba950bed410a12c1294df28ed672f50c24297de
290968,V3_SWAP_EXACT_IN,0xeb5baae9576cea96ad468062a369bdaa2c5ed0a32e5cb5e1dac0b6e0e43c507d,0x0000000000000000000000000000000000000002,1000000000000000000,None,7744132,None,True,0x9dcd367e2afa8d6e5d6cf0306094e3eb7bbaaf4d,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xebed4ae2f4158fd9a16bbcda446a9410a56e3151
290969,V3_SWAP_EXACT_IN,0xf805624fa76ccdf5a015257bc43efd0ca8523ddd570617abc5d61e7ebb3101a5,0x0000000000000000000000000000000000000002,900000000000000000000000,None,89385817684522659,None,True,0x2c2ee7522bbd2b7ee75987cc56fe6f5191d199ff,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x422d476255f5fd8c125ca90793b492ad5f31e7e2


In [57]:
engine = create_engine(postgresql_uri_mp)

df_joined.to_sql("swap_limit_price", engine, if_exists="replace", index=False, chunksize=10_000, method="multi", dtype=String)

290971